In [1]:
print("Hello World")
DEBUG = False

Hello World


In [2]:
import spacy
import os
import subprocess
import langid
import csv, json, ast, requests
import numpy as np
import datetime
import string
from collections import Counter
import copy
import requests

In [3]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import text as sklearn_text
from scipy.cluster.hierarchy import cophenet, dendrogram
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import fcluster
from scipy import cluster
from sklearn.neighbors import kneighbors_graph
from scipy.spatial.distance import pdist, squareform
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [4]:
from functools import reduce

In [5]:
#Tokenizing Methods
from nltk import word_tokenize

In [6]:
#Import stop words
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as en_stopwords

In [7]:
#Import matplotlib for visualization of t-SNE
from matplotlib import colors as mcolors
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import mplcursors


In [8]:
#Vadersentiment 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [9]:
import shutil

In [54]:
from sklearn.metrics.pairwise import cosine_similarity as cos_sim

In [10]:
#..to be wrapped using .sh script
print("All Imports Completed!")

All Imports Completed!


In [11]:
### .shell script for downloading glove embeddings
# download word vector
"""
if [ ! -e $WORDFILE ]; then
wget http://nlp.stanford.edu/data/glove.840B.300d.zip
unzip glove.840B.300d.zip -d $DATADIR
rm glove.840B.300d.zip
fi
"""

'\nif [ ! -e $WORDFILE ]; then\nwget http://nlp.stanford.edu/data/glove.840B.300d.zip\nunzip glove.840B.300d.zip -d $DATADIR\nrm glove.840B.300d.zip\nfi\n'

In [12]:
## input
# word vector file, can be downloaded from GloVe website
wordfile = 'data/glove.840B.300d.txt'
# each line is a word and its frequency; can be made from corpus
weightfile = 'auxiliary_data/enwiki_vocab_min200.txt' 
weightpara = 1e-3 # the parameter in the SIF weighting scheme, usually in the range [3e-5, 3e-3]
rmpc = 1 # number of principal components to remove in SIF weighting scheme
output_folder = 'output_docs/'
filing_list = 'filing_list.txt'
output_jsons_path = 'all_slices_json.txt'

In [72]:
#DEBUG = True
DEBUG = False
import params

def load_params(params, wordfile, weightfile, weightpara, rmpc, output_folder, filing_list, output_jsons_path, docket_num):
    cwd = os.getcwd()+'/'
    params.wordfile = cwd+wordfile
    params.weightfile = cwd+weightfile
    params.weightpara = weightpara
    params.rmpc = rmpc
    params.output_folder = cwd+output_folder
    params.filing_list = filing_list
    params.output_jsons_path = cwd+output_jsons_path
    params.is_output_jsons = False
    params.output_jsons = None
    params.stopwords = en_stopwords
    params.my_model = None
    params.my_docket_num = docket_num

In [73]:
#TODO: Run on Stanford Cloud Service
#TODO: Try iterator pattern for better performance
#Note:
#          returns only the newly updated list of files and jsons.
#          need to read from files directly to obtain the whole set;
#           or set is_new_filing_list=True.

def get_FCC_docs_(myAPIKey, params, is_new_filing_list=False, is_new_folder=False, attachments=False, 
                  start_ind=0, end_ind=-1):
    def unpopulate_filing_list(params):
        with open(filing_list_path, 'w') as fl:
            fl.write('')
            
    APIUrl = "https://publicapi.fcc.gov/ecfs/"
    raw_APIKey = myAPIKey
    """
    try: #check for api key
        raw_APIKey = sys.argv[1]
    except (ValueError, IndexError):
        raise ValueError("Invalid API key")
    """
    APIKey = "&api_key="+raw_APIKey
    
    #forming query
    #CHECK SAFETY OF my_docket_num with RegEx.
    
    url = APIUrl + 'filings?proceedings.name='+ params.my_docket_num +'&type=downloadplan'+APIKey
    if(DEBUG): print(url)
    response = requests.get(url)
    parsed = response.json()
    
    if os.path.exists(params.output_folder):
        if(is_new_folder):
            shutil.rmtree(params.output_folder)
            os.makedirs(params.output_folder)
    else:
        os.makedirs(params.output_folder)
    
    #make filing list
    filing_list_path = params.output_folder+params.filing_list
    if not os.path.exists(filing_list_path) or is_new_filing_list:
        unpopulate_filing_list(params)
        existing_file_names = []
    else:
        with open(params.output_folder+params.filing_list, 'r') as fd:
            existing_file_names = [file_name.strip() for file_name in fd.readlines()]
        
    #get download plan in json
    downloadplan_path = params.output_folder+'downloadplan.txt'
    if not os.path.exists(downloadplan_path):
        with open(downloadplan_path, 'w') as dpp:
            dpp.write('')
    
    #get parsed documents
    parsed_list = []
    for doc in parsed['aggregations']['download_plan']['buckets']:
        parsed_list.append(doc)
        
    my_file_slice_key_list = []
    my_file_slice_json_list = []
    doc_count = 0
    

    for doc in parsed_list[start_ind:end_ind]:
        if(DEBUG): 
            print(doc_count)
            #if(doc_count == 3): break
                
        doc_count += 1
        file_slice = str(doc['key'])
        #print(json.dumps(doc, indent=4, sort_keys=True))
                
        if doc['suggested_api_call']:
            for local_count, api_call in enumerate(doc['suggested_api_call']):
                file_slice_key = file_slice + str(local_count)+'.txt'
                
                #avoiding requesting duplicate files
                if(DEBUG): print(existing_file_names)
                if file_slice_key not in existing_file_names:
                    my_file_slice_key_list.append(file_slice_key)
                else:
                    break;
                
                #API Call
                api_call_key = api_call+APIKey
                print('-->' + api_call_key)
                slice_req = requests.get(api_call_key, stream=True)
                
                #overwrite even if it exists
                file_slice = open(params.output_folder+file_slice_key, 'wb+')
                for chunk in slice_req.iter_content(chunk_size=256):
                    if(chunk):
                        file_slice.write(chunk)
                
                ##Checking Rate Limit
                file_slice.seek(0)
                temp_read = json.load(file_slice)
                file_slice.close()
                if(DEBUG): print(temp_read.keys()) #if request limit exceeds, return.
                if('error' in temp_read.keys()):
                    print('{0}: request limit exceeded. Error: {1}'.format(count, temp_read['error']))
                    return my_file_slice_key_list, my_file_slice_json_list
                
                
                my_file_slice_json_list.append(temp_read)
                
                #recording the name of the file
                if(is_new_filing_list):
                    with open(filing_list_path, 'w') as fd:
                        fd.write(file_slice_key+'\n')
                else:
                    with open(filing_list_path, 'a') as fd:
                        fd.write(file_slice_key+'\n')
    
    #dump list of json into one file so that we can read directly without looping through individual files.
    with open(params.output_jsons_path, 'w') as fd:
        fd.write(json.dumps(my_file_slice_json_list))
        
    params.output_jsons = my_file_slice_json_list
    params.output_file_name_list = my_file_slice_key_list

    params.is_output_jsons = True
    return my_file_slice_key_list, my_file_slice_json_list

In [18]:
#IDEA: Build a class for text-tree that has 
#raw, rare, filtered, doc_index, file_index
def load_params_json(params):
    try:
        with open(params.output_jsons_path, 'r') as fd:
            params.output_jsons = json.load(fd)
            params.is_output_jsons = True
        return True
    except:
        return False

In [19]:
#load_params_json(params)

False

In [21]:
#all_slices_json_f = open(output_jsons, 'r')
#all_slices_json = json.load(all_slices_json_f)
#print(file_slice_json[0]['filings'][1])

#Turn off local reading of jsons by setting:
# params.is_output_jsons = False
# TODO: pre-load all jsons from separate jsons in output_folder into one list
def wrap_data(params, ignore_docs=False):
    new_data = []
    
    #if doesn't exist, load the most recent (if exists)
    try:
        if not params.is_output_jsons:
            load_params_json(params)
        print('Using most recent successful case of json production.')
    except:
        pass
    
    if params.is_output_jsons:
        total_count = 0
        for doc_index, doc_json in enumerate(params.output_jsons):
            
            if('filings' not in doc_json):
                print('!! filing not present..')
                print(doc_json.keys())
            
            if(DEBUG): print(doc_index, doc_json.keys())

            for file_index, data in enumerate(doc_json['filings']):
                if(file_index % 1000 == 0): print(file_index)
                wrapped_data = {}  

                keys = data.keys()
                #total filing index
                wrapped_data['tag'] = total_count
                total_count += 1
                
                #submission type
                wrapped_data['submission_type'] = data['submissiontype']['description']
                #wrapped_data['confirmation_number'] = data['confirmation_number']
                wrapped_data['id_submission'] = data['id_submission']
                #wrapped_data['address'] = data['addressentity']
                #wrapped_data['data_received'] = data['data_received']
                #wrapped_data['contact_email'] = data['contact_email']
                #wrapped_data['filer'] = data['filer']
                #wrapped_data['filing_status'] = data['filing_status']
                wrapped_data['is_text_data'] = False
                wrapped_data['is_doc_data'] = False
                wrapped_data['is_public_notice'] = False
                ##Importing text/documents/attachments
                
                ###text data
                if(wrapped_data['submission_type'] == 'COMMENT'):
                    if('text_data' in data):
                        wrapped_data['is_text_data'] = True
                        wrapped_data['filtered_text_data'] = my_filter_text(params, data['text_data'], full_filter_text)
                        wrapped_data['rare_text_data'] = my_filter_text(params, data['text_data'], raw_filter_text)
                        wrapped_data['raw_text_data'] = my_filter_text(params, data['text_data'])
                        wrapped_data['doc_index'] = doc_index
                        wrapped_data['file_index'] = file_index
                        
                    if(ignore_docs==False and 'document' in data):
                        wrapped_data['is_doc_data'] = True
                        doc_text_list = []
                        doc_raw_text_list = []
                        #doc_embedding_list = []
                        for doc in data['document']:
                            #forming query
                            url = 'https://ecfsapi.fcc.gov/file/'+ wrapped_data['id_submission'] + doc['filename'].replace(' ', '%20')
                            print(url)
                            response = requests.get(url)
                            parsed = response.text
                            
                            filtered_text = filter_text(parsed, en_stopwords_list)
                            doc_text_list += filtered_text
                            doc_raw_text_list.append(parsed)
                            #doc_embedding_list.append(apply_embedding(model, filtered_text))
                        
                        wrapped_data['doc_text_list'] = doc_text_list
                        wrapped_data['doc_raw_text_list'] = doc_raw_text_list
                        #wrapped_data['doc_embedding_list'] = doc_embedding_list
                
                if(wrapped_data['submission_type'] == 'PUBLIC NOTICE'):
                    wrapped_data['is_public_notice'] = True
                
                if(wrapped_data['is_text_data']):
                    new_data.append(wrapped_data)
                
        return new_data
    else:
        #the connection failed or the filing scrapping was interrupted.
        #TODO: directly call from all files in output_folder.
        print('Please re-complete the filing scrapping at this current version.')
        return;


In [25]:
import hashlib
def getMD5_from_file(filename, block_size=4096):
    md5 = hashlib.md5()
    try:
        with open(filename, 'rb') as f:
            while True:
                data = f.read(block_size)
                if not data:
                    break
                md5.update(data)
    except IOError:
        print('File' + filename + 'Not found!' )
        return None
    except:
        return None
    return md5.hexdigest()

def getMD5_from_text(text):
    md5 = hashlib.md5()
    md5.update(text.encode('utf-8'))
    return md5.hexdigest()

In [27]:
import text_chunk #THIS IS UNDER CONSTRUCTION
import collections

def extract_text_data(wrapped_data_list, sub_type='COMMENT', allow_duplicate=False, len_lim=99999):
    #my_chunk = text_chunk.text_chunk()
    selected_wrapped_data = []
    raw_text_list = []
    rare_text_list = []
    filtered_text_list = []
    my_hash_countable = {}
    my_hash_rawtable = {}
    
    #!could handle all sub_types: TEXT_ONLY, DOC_ONLY, ALL
    for cnt, wrapped_data in enumerate(wrapped_data_list):
        #if(cnt == 3): break
        if(wrapped_data['submission_type'] == sub_type):
            if(wrapped_data['is_text_data']):
                #USE staticmethod?
                #my_chunk.temp_wrapped_data = wrapped_data
                #my_chunk.temp_allow_duplicate = allow_duplicate
                #no_duplicate = my_chunk.add_all_dict()
            
                #We can be as strict as to check raw_text, or as loose as filtered_text. 
                #returns: whether if the text passed the duplicate test.
                test_sent = wrapped_data['rare_text_data']
                my_hash = getMD5_from_text(test_sent)
                
                if allow_duplicate or (my_hash not in my_hash_countable and len(test_sent) < len_lim):
                #kinda NOR operation
                #can't directly compare strings.
                #if allow_duplicate or test_sent.split() not in [raws.split() for raws in raw_text_list]:
                    #print(raw_text_list[-1].split() == test_sent.split())
                   
                    #print(len(raw_text_list))
                    
                    #print(test_sent)
                    #print(my_hash)
                    raw_text_list.append(wrapped_data['raw_text_data'])
                    rare_text_list.append(wrapped_data['rare_text_data'])
                    filtered_text_list.append(wrapped_data['filtered_text_data'])
                    selected_wrapped_data.append(wrapped_data)
                    
                    if(my_hash not in my_hash_countable):
                        my_hash_countable.update({my_hash:1})
                        my_hash_rawtable.update({my_hash: wrapped_data['raw_text_data']})
                    else:
                        my_hash_countable[my_hash] += 1
                        
            #TODO: filtering doc text need to be fixed in wrapping.
            """
            if(wrapped_data['is_doc_data']):
                all_docs = ''
                for doc in wrapped_data['doc_text_list']:
                    all_docs += doc
                text_chunk.add_raw_text(all_docs)
            """
    if(allow_duplicate):
        my_cnt = 0
        duplicate_list=[]
        for k,v in sorted(my_hash_countable.items(), key= lambda kv: kv[1], reverse=True): 
            if(my_cnt == 3000 or v == 1):
                break
            print('Top {0} comment'.format(my_cnt+1))
            print('_hash name: {0}\n_repeat_count: {1}\n_text_data: \n{2}\n\n'.format(k, v, my_hash_rawtable[k]))
            #print(k,v, my_hash_rawtable[k])
            my_cnt += 1
            duplicate_list.append({'text':my_hash_rawtable[k], 'count':v})
            
            
        #constantly updating bot_detected_list
        #with open('bots_raw_texts.json', 'w') as bot_f:
        #    json.dump(duplicate_list, bot_f)
            
    print(len(raw_text_list))
    #print(raw_text_list)
    my_text_chunk = [raw_text_list, rare_text_list, filtered_text_list]
    return selected_wrapped_data, my_text_chunk

In [29]:
with open('bots_raw_texts.json', 'r') as bot_f:
    my_json_list = json.load(bot_f)
    print(len(my_json_list))
    for my_json in my_json_list:
        print(my_json)

3000
{'text': 'I was outraged by the Obama/Wheeler FCC\'s decision to reclassify the Internet as a regulated "public utility" under a Depression-era law written for the old Ma Bell telephone monopoly.\n\nGovernment utility regulation of the Internet risks devastating private investment, undermining competition, and stalling innovation. It also puts consumers at serious risk of being hit with a new "broadband tax" to cover the lack of private sector investment due to these regulations.\n\nThe liberal extremist groups that ginned up fake support for reclassification include the group Free Press, which was cited 62 times in the Title II order.\n\nFree Press was founded by ultraliberal college professor Robert McChesney who has admitted: "At the moment, the battle over network neutrality is not to completely eliminate the telephone and cable companies. We are not at that point yet. But the ultimate goal is to get rid of the media capitalists in the phone and cable companies and to divest t

{'text': 'I support strong Net Neutrality backed by Title II oversight of ISPs. The internet should be treated as a utility and the ISP providers should not be able to regulate the content and speed for internet users based on their preference nor their business subsidiaries. Moreover, search engines should also be subject to these regulations as well, so they are not attempting to control content internet users are privy.', 'count': 8}
{'text': 'I support Title II, preserve net neutrality.', 'count': 8}
{'text': 'Destroying Net Neutrality will not only hurt innovation, but destroy the freedom of the internet as we know it. The fact that it is called "restoring internet freedom" is disgraceful and misleading to the American people. If the US wants to remain a leader in technology, we must keep the internet truly free and continue to treat it as the utility that it is. We must keep Net Neutrally - the future of our country depends on it.', 'count': 8}
{'text': 'I wanted to let Mr. Pai k

In [32]:
#return tf-idf matrix and words in tf-idf
#IDEA: could pre-define unigram/bigram vectorizer out of function
def get_tfidf_matrix(text_list, ngram=1, stopwords=None):
    def get_doc_iterator(doc_token_collection, by_sentence=False, indiv_tokens=False, stemmed=False,
                    stopwords=None, filter_fn = lambda x: True, get_key= False, skip_form_letters=False,
                    condense_form_letters=False, get_names=False):
        for sent in doc_token_collection:
            tokens = word_tokenize(sent)
            yield " ".join(tokens)
            #print(tokens, sent.split())  #same thing
            #yield " ".join(sent.split())
            
    
    #DESCRIBING TF-IDF
    #sublinear_tf: Apply sublinear tf scaling, i.e. replace tf with 1 + log(tf).
    #smooth_idf: add-one smoothing.
    #use_idf: Enable inverse-document-frequency reweighting.  
    tfidf_vectorizer = sklearn_text.TfidfVectorizer(min_df=2, stop_words = stopwords, strip_accents = 'unicode', 
                                                lowercase=True, ngram_range=(1,ngram), norm='l2', smooth_idf=True, 
                                                sublinear_tf=False, use_idf=True)
    sklearn_doc_iterator = get_doc_iterator(text_list, indiv_tokens=False, stemmed=False, condense_form_letters=True)
    tfidf_matrix = tfidf_vectorizer.fit_transform(sklearn_doc_iterator)
    words_tfidf = np.array(tfidf_vectorizer.get_feature_names())
    return tfidf_matrix, words_tfidf

In [36]:
#principal components
#Return transformed mat and hierarchy cluster using linkage method.
def reduce_topic_dim(mat, n=100):
    topic_dimensionality = n
    tsvd = TruncatedSVD(topic_dimensionality)
    
    tsvd.fit(mat)
    V_tf = tsvd.components_
    U_tf = tsvd.transform(mat)
    print(V_tf.shape)
    print(V_tf[0])
    print(U_tf.shape)
    print(U_tf[0])
    X = U_tf
    
    return X

In [37]:
def get_hier_cluster_of(X, plot_name=None):
    """
    a measure of how faithfully a dendrogram preserves the pairwise 
    distances between the original unmodeled data points. 
    """
    Z = cluster.hierarchy.linkage(X, 'ward')
    full_dendrogram(Z, plot_name=plot_name)
    c, coph_dists = cophenet(Z, pdist(X))
    print("cophenetic constant is {0}".format(c))
    
    return Z

In [38]:
"Hierarchical clustering:"

def full_dendrogram(Z, plot_name='Hierarchical Clustering Dendrogram'):
    plt.figure(figsize=(10, 6))
    plt.title(plot_name)
    plt.xlabel('sample index')
    plt.ylabel('distance')
    dendrogram(
        Z,
        truncate_mode='lastp',  # show only the last p merged clusters
        p=20,  # show only the last p merged clusters
        leaf_rotation=90.,  # rotates the x axis labels
        leaf_font_size=8.,  # font size for the x axis labels
    )
    plt.show()
    
    

In [41]:
def tsne_coords(
        mat, dim=2, suppress=True):     
    t_start = time.process_time()
    
    # Recommended reduction via PCA or similar:
    #n_components = 50 if mat.shape[1] >= 50 else mat.shape[1]
    #dimreduce = PCA(n_components=n_components)
    #mat = dimreduce.fit_transform(mat) 
    
    # t-SNE:
    final_dimensions = dim
    tsne = TSNE(n_components=final_dimensions, random_state=1, n_iter=2000) #2000 is better, perplexity=50
    np.set_printoptions(suppress=suppress)    
    tsnemat = tsne.fit_transform(mat) 
    
    print(' t-SNE matrix forming took '+ str(time.process_time-t_start) + 'seconds')
    return tsnemat


def tsne_visualize(xyzcoord,
        rownames,
        colors=None,
        output_filename=None,
        figheight=80,
        figwidth=100, 
        plot_name=None):

    indices = list(range(len(rownames)))
    
    # Colors:
    if not colors:
        colors = ['black' for i in indices]   
    
    # Plot values:
    vocab = np.array(rownames)[indices]
    
    # Plotting:
    coords = list(zip(*xyzcoord))
    
    if(len(xyzcoord[0]) == 2):
        xvals, yvals = coords[0], coords[1]

        fig, ax = plt.subplots(nrows=1, ncols=1)
        #fig.set_figheight(max(xvals))
        #fig.set_figwidth(max(yvals))
        
        ax.plot(xvals, yvals, marker='', linestyle='')
        
        # Text labels:
        for word, x, y, color in zip(vocab, xvals, yvals, colors):
            ax.annotate(word, (x, y), fontsize=8, color=color)
   
    if(len(xyzcoord[0]) == 3):
        xvals, yvals, zvals = coords[0], coords[1], coords[2]
        #fig, ax = plt.subplots(111, projection='3d')
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        #fig.set_figheight(max(xvals))
        #fig.set_figwidth(max(yvals))
        #fig.set_figwidth(max(zvals))

        
        ax.scatter(xvals, yvals, zvals, c=colors, marker='x')
        # Text labels:
        #for word, x, y, z, color in zip(vocab, xvals, yvals, zvals, colors):
        #   ax.scatter(x,y,z, alpha=0.8, c=color, edgecolors='none', s=30, label=word)

    # Output:
    if output_filename:
        pass
        #TODO: Invalid RGBA argument??
        #plt.savefig(output_filename, bbox_inches='tight')

    if(plot_name):
        plt.title(plot_name)
        
    mplcursors.cursor(hover=True)
    plt.show() 

In [45]:
import random
def get_color_data(labels, color_index_hash=None):
    
    if(color_index_hash == None): color_index_hash = lambda x,length: (x%length)
    else: 
        pass
        #color_index_hash = lambda x,length: (x*100+977)%length
        
    raw_color_keys = list(set(mcolors.CSS4_COLORS.keys()))
    #shuffled_raw_color_keys = random.sample(raw_color_keys, len(raw_color_keys))
    
    color_data = [raw_color_keys[color_index_hash(index,len(raw_color_keys))] for index in labels]
    return color_data, raw_color_keys

In [47]:
def plot_tsne(color_data, tsnemat, plot_name=None):
    t_start = time.process_time()
    my_coords = []
    for coord in tsnemat:
        my_coords.append(coord)
            
    my_coords = [np.array(xy) for xy in my_coords]
    #xvals=normalized_converted_timestamps*scaling_factor #should be real time information from data pull.
    tsne_visualize(my_coords, ['x']*len(tsnemat), colors=color_data, 
                   output_filename=plot_name, plot_name=plot_name)
    
    print(' took '+ str(time.process_time() - t_start) + 'seconds')
    return my_coords

In [49]:
def get_weighted_average(model, matrix, words, weight_fn=None, size_of_emb=300):
    n_samples = matrix.shape[0]
    embs = np.zeros((n_samples, size_of_emb))
    print(embs.shape)
    not_found_words = {}
    for cnt, row in enumerate(matrix):
        #if(cnt == 6): break
        k = matrix[cnt,:]
        
        weights = np.array(k.data)
        weights = np.array([1 for i in range(len(k.data))])
        
        one_indices = k.indices
        #one_indcies = [1 for i in range(len(k.indices))]
        
        g_embs = []
        g_weights = []
        for cnt2, index in enumerate(one_indices):
            my_word = words[index]
            try:
                g_embs.append(np.array(model[my_word]))
                g_weights.append(np.array(weights[cnt2]))
            except:
                if my_word not in not_found_words:
                    print('word Not Found: {0}'.format(my_word))
                    not_found_words[my_word] = 1
                else:
                    not_found_words[my_word] += 1
                    if(not_found_words[my_word] % 100 == 0):
                        print('word Not Found: {0} hit {1} times'.format(my_word, not_found_words[my_word]))
            
        #print(weights.shape, g_embs.shape)
        g_embs = np.array(g_embs)
        g_weights = np.array(g_weights)
        
        if(len(g_embs) == 0):
            #print("wtf")
            #print(g_weights.dot(g_embs))
            print(cnt)
            embs[cnt, :] = np.array([0 for i in range(size_of_emb)])
        else:
            #print(g_embs.shape)
            #print(g_weights.dot(g_embs))
            embs[cnt, :] = g_weights.dot(g_embs) / np.count_nonzero(g_weights)
        #print(embs[cnt,:])
        if(cnt==847): print(embs[cnt,:])
    return embs

In [50]:

def same_doc(doc1, doc2):
    for index, word in enumerate(doc1.split()):
        if(word != doc2.split()[index]):
            #print(word)
            break
        else:
            #print(word)
            continue
    return True

#print(my_text_chunk[0][0] in [my_text_chunk[0][0], my_text_chunk[0][2]])
#print(my_text_chunk[0][0].split() == my_text_chunk[0][2].split())

In [53]:
def load_model(wordfile):
    print('Loading Model from {0}'.format(wordfile))
    f = open(wordfile, 'r')
    model = {}
    for cnt, line in enumerate(f):
        #if(cnt == 100): break
        splitLine = line.split()
        
        word_ind=0
        while(True):
            val = splitLine[word_ind]
            try:
                float(val)
                break
            except:
                word_ind += 1
                
        word = ''.join(splitLine[:word_ind])
        #print(word)
        try:
            embedding = np.array([float(val) for val in splitLine[word_ind:]])
        except:
            print('EMBEDDING MISS')
            print(word, embedding)
            break
        model[word] = embedding
    print('Done', len(model), ' words are loaded.')
    return model
#glove_model = load_model(params)

In [55]:
def compute_pc(X, npc=1):
    svd = TruncatedSVD(n_components=npc, n_iter=10, random_state=0)
    svd.fit(X)
    components_ = svd.components_
    print(X.shape)
    return X, components_

In [56]:
def remove_pc(X, npc=1):
    new_X, pc = compute_pc(X, npc)
    #XX = X - X(U^TU)
    #print(new_X.shape,X.shape)
    if npc == 1:
        XX = X - X.dot(pc.transpose()) * pc
    else:
        XX = X - X.dot(pc.transpose()).dot(pc)
    return XX

In [57]:
def do_SIF_embedding(model, my_matrix, words, weight_fn = None, emb_dim=300, npc=1):
    embs = get_weighted_average(model, my_matrix, words, size_of_emb=emb_dim)
    print(embs.shape)
    print(embs)
    if npc > 0:
        embs = remove_pc(embs, npc)
    print(embs.shape)
    return embs

In [58]:
def do_average(emb):
    return reduce(lambda x,y: x+y, emb)/len(emb)

In [59]:
#use lambda
def sent2emb(model, sent, apply_fn=None):
    sent = sent.lower().split()
    sent_vec = []
    for cnt, word in enumerate(sent):
        try:
            emb = model[word]
            sent_vec.append(np.array(emb))
            print(word)
        except:
            print('No Embedding Found in the Model.')
    sent_vec = np.array(sent_vec)
    if(apply_fn):
        return apply_fn(sent_vec)
    else:
        return sent_vec

In [60]:
#params passed on due to stopwords
def full_filter_text(params, text):
    #TODO: ? and ! may be needed in sentiment analysis
    temp_text = text.lower().replace('/', '').replace('!','').replace('?', '').replace(':', '').replace(';', ' ').replace('\r', '').replace('\n', ' ').replace('.', '').replace(',', '').replace('"', '').replace("'", '').replace('(', '').replace(')', '').replace('{', '').replace('}', '').replace('[', '').replace(']', '').replace("can't", "can not").replace("shouldn't", "should not").replace("couldn't", "could not").replace("wouldn't", "would not").replace("wasn't", "was not").replace("don't", "do not").replace("doesn't", "does not").replace("didn't", "did not").replace("hasn't", "has not").replace("haven't", "have not").replace("'ll", " will").replace("I'm", "I am").replace("he's", "he is").replace("she's", "she is").replace("it's", "it is")
    tokens = ' '.join([w for w in temp_text.split(' ') if w.isalpha()]).split(' ')
    if(params.stopwords):
        filtered_tokens = [w for w in tokens if not w in params.stopwords]
    return ' '.join(filtered_tokens)

In [62]:
def raw_filter_text(params, text):
    temp_text = text.lower().replace('/', '').replace('!','').replace('?', '').replace(':', '').replace(';', ' ').replace('\r', '').replace('\n', ' ').replace('.', '').replace(',', '').replace('"', '').replace("'", '').replace('(', '').replace(')', '').replace('{', '').replace('}', '').replace('[', '').replace(']', '').replace("can't", "can not").replace("shouldn't", "should not").replace("couldn't", "could not").replace("wouldn't", "would not").replace("wasn't", "was not").replace("don't", "do not").replace("doesn't", "does not").replace("didn't", "did not").replace("hasn't", "has not").replace("haven't", "have not").replace("'ll", " will").replace("I'm", "I am").replace("he's", "he is").replace("she's", "she is").replace("it's", "it is")
    tokens = ' '.join([w for w in temp_text.split(' ') if w.isalpha()]).split(' ')
    return ' '.join(tokens)

In [63]:
def my_filter_text(params, text, filter_fn=None):
    if(filter_fn):
        return filter_fn(params, text)
    else:
        return text

In [64]:
def doc2bigram(params, documents, filter_fn=None):
    from gensim.models import Phrases
    from gensim.models.phrases import Phraser

    if(filter_fn):
        sentence_stream = [filter_fn(params, doc).lower().split(' ') for doc in documents]
    else:
        sentence_stream = [doc.lower().split(' ') for doc in documents]

    
    bigram = Phrases(sentence_stream, min_count=1, threshold=2)
    bigram_phraser = Phraser(bigram)

    #print(bigram_phraser)
    
    result_tokens = []
    for sent in sentence_stream:
        tokens_ = bigram_phraser[sent]
        result_tokens.append(tokens_)
    return result_tokens

In [65]:
def do_visualize(glove_model, rare_unigram_mat_no_stop, rare_unigram_words_no_stop, plot_name='',k=10, n=100):
    #test_X_tfidf_rare_uni_no_stop = reduce_topic_dim(rare_unigram_mat_no_stop)
    test_X_tfidf_rare_uni_no_stop = reduce_topic_dim(rare_unigram_mat_no_stop, n)
    test_Z_tfidf_rare_uni_no_stop = get_hier_cluster_of(test_X_tfidf_rare_uni_no_stop)

    test_X_sif_rare_uni_no_stop = do_SIF_embedding(glove_model, rare_unigram_mat_no_stop, rare_unigram_words_no_stop)
    test_Z_sif_rare_uni_no_stop = get_hier_cluster_of(test_X_sif_rare_uni_no_stop)
    
    #k=20
    test_tsnemat_tfidf_rare_uni_no_stop = tsne_coords(test_X_tfidf_rare_uni_no_stop)
    test_clusters_tfidf_uni_no_stop = fcluster(test_Z_tfidf_rare_uni_no_stop, k, criterion='maxclust')
    test_color_data_tfidf_uni_no_stop = get_color_data(test_clusters_tfidf_uni_no_stop)
    test_xy_tfidf_uni_no_stop = plot_tsne(test_color_data_tfidf_uni_no_stop, test_tsnemat_tfidf_rare_uni_no_stop, plot_name = 'tfidf'+plot_name)

    test_tsnemat_sif_rare_uni_no_stop = tsne_coords(test_X_sif_rare_uni_no_stop)
    test_clusters_sif_uni_no_stop = fcluster(test_Z_sif_rare_uni_no_stop, k, criterion='maxclust')
    test_color_data_sif_uni_no_stop = get_color_data(test_clusters_sif_uni_no_stop)
    test_xy_sif_uni_no_stop = plot_tsne(test_color_data_sif_uni_no_stop, test_tsnemat_sif_rare_uni_no_stop, plot_name='sif'+plot_name)

    #print(test_xy_sif_uni_no_stop)

In [67]:
#########DEMO#########

In [ ]:
## input
# word vector file, can be downloaded from GloVe website
wordfile = 'data/glove.840B.300d.txt'
# each line is a word and its frequency; can be made from corpus
weightfile = 'auxiliary_data/enwiki_vocab_min200.txt' 
weightpara = 1e-3 # the parameter in the SIF weighting scheme, usually in the range [3e-5, 3e-3]
rmpc = 1 # number of principal components to remove in SIF weighting scheme
output_folder = 'output_docs/'
filing_list = 'filing_list.txt'
output_jsons_path = 'all_slices_json.txt'

In [83]:
import params
def init_param(docket_number):
    p = params.params()
    load_params(p, wordfile, weightfile, weightpara, rmpc, output_folder, filing_list, output_jsons_path, docket_number)
    return p

In [85]:
#demo
#Load params
import params

#17-108 
#Freedom of Internet
params_17_108 = init_param('17-108')

#02-6
#In the Matter of Schools and Libraries Universal Service Support Mechanism
params_026 = init_param('02-6')

#06-229
#In the Matter of Implementing a nationwide, Broadband, Interoperable Public Safety Network in the 700 MHz Band. Development of Operational, Technical and Spectrum Requirements for Meeting Federal, State and Local Public Safety Communication .
params_06229 = init_param('06-229')

#92-235
#REPLACEMENT OF PART 90 BY PART 88 TO REVISE THE PRIVATE LAND MOBILE RADIO SERVICES AND MODIFY THE POLICIES GOVERNING THEM
params_92235 = init_param('92-235')

#92-235
#In the Matter of Improving Public Safety Communications in the 800 MHz Band
params_0255 = init_param('02-55')



In [71]:
#Load Word-Vector Model such as word2vec or GloVe
if params.my_model == None:
    glove_model = load_model(params.wordfile)

params.my_model = glove_model

params.is_output_jsons

False

In [ ]:
params.is_output_jsons

In [ ]:
print(speed_bump)

In [ ]:
myAPIKey = 'wd73djRi6qOZHUHIqCNbuzXo890BrzbrfmdWLfrr'

print(params.is_output_jsons)
test_file_slice_list, test_file_slice_json = get_FCC_docs_(myAPIKey, params, is_new_filing_list=True, is_new_folder=False, attachments=False, start_ind=20, end_ind=23)
print(params.is_output_jsons)


In [ ]:
test_wrapped_data_list = wrap_data(params, ignore_docs=True)
print(len(test_wrapped_data_list))

In [ ]:
test_selected_wrapped_data, test_my_text_chunk = extract_text_data(test_wrapped_data_list, allow_duplicate=False, len_lim=500)

print(len(test_selected_wrapped_data))
print(len(test_my_text_chunk[0]))

#Constructing TFIDF matrix from 'rare text' with no stopword
rare_unigram_mat_no_stop, rare_unigram_words_no_stop = get_tfidf_matrix(test_my_text_chunk[1], ngram=1,stopwords=None)

print(rare_unigram_mat_no_stop.shape)

#Constructing SIF matrix from 'rare text' with no stopword
rare_unigram_mat_stop, rare_unigram_words_stop = get_tfidf_matrix(test_my_text_chunk[1], ngram=1)

print(rare_unigram_mat_stop.shape)


In [ ]:
ncnt = 300
for acnt in range(10):
    print(' batch '+str(acnt+1))
    for pcnt in range(15):
        print('comment #{0}'.format(pcnt+1))
        print(test_my_text_chunk[0][ncnt*15])
        ncnt += 1
    print('\n')

In [ ]:
#do_visualize(params.my_model, rare_unigram_mat_no_stop, rare_unigram_words_no_stop, plot_name='_no_stop', k=11, n=200)


In [ ]:
"""
#demo
#Load params
import params
params = params.params()

## input
# word vector file, can be downloaded from GloVe website
wordfile = 'data/glove.840B.300d.txt'
# each line is a word and its frequency; can be made from corpus
weightfile = 'auxiliary_data/enwiki_vocab_min200.txt' 
weightpara = 1e-3 # the parameter in the SIF weighting scheme, usually in the range [3e-5, 3e-3]
rmpc = 1 # number of principal components to remove in SIF weighting scheme
output_folder = 'output_docs/'
filing_list = 'filing_list.txt'
output_jsons_path = 'all_slices_json.txt'

#load_params(params, wordfile, weightfile, weightpara, rmpc, output_folder, filing_list, output_jsons_path)

if params.my_model == None:
    glove_model = load_model(params.wordfile)
    params.my_model = glove_model

myAPIKey = 'wd73djRi6qOZHUHIqCNbuzXo890BrzbrfmdWLfrr'

print(params.is_output_jsons)
test_file_slice_list, test_file_slice_json = get_FCC_docs_(myAPIKey, params, is_new_filing_list=True, is_new_folder=False, attachments=False, start_ind=3, end_ind=5)
print(params.is_output_jsons)
test_wrapped_data_list = wrap_data(params, ignore_docs=True)

print(len(test_wrapped_data_list))

test_selected_wrapped_data, test_my_text_chunk = extract_text_data(wrapped_data_list, allow_duplicate=False)

print(len(test_selected_wrapped_data))
print(len(test_my_text_chunk[0]))
rare_unigram_mat_no_stop, rare_unigram_words_no_stop = get_tfidf_matrix(my_text_chunk[1], ngram=1,stopwords=None)
rare_unigram_mat_stop, rare_unigram_words_stop = get_tfidf_matrix(my_text_chunk[1], ngram=1)

print(rare_unigram_mat_no_stop.shape)
print(rare_unigram_mat_stop.shape)

#do_visualize(rare_unigram_mat_no_stop, rare_unigram_words_no_stop, plot_name='_no_stop', k=11, n=200)
#do_visualize(rare_unigram_mat_stop, rare_unigram_words_stop, k=11)

"""

In [ ]:
#defining tfidf - part1
t_start = time.process_time()

test_X_tfidf_rare_uni_no_stop1 = reduce_topic_dim(rare_unigram_mat_no_stop, n=50)
print(' took '+ str(time.process_time() - t_start) + 'seconds')

test_X_tfidf_rare_uni_no_stop2 = reduce_topic_dim(rare_unigram_mat_no_stop, n=500)
test_Z_tfidf_rare_uni_no_stop = get_hier_cluster_of(test_X_tfidf_rare_uni_no_stop1)

print(' took '+ str(time.process_time() - t_start) + 'seconds')

In [ ]:
#defining sif - part2-final
t_start = time.process_time()

test_X_sif_rare_uni_no_stop = do_SIF_embedding(glove_model, rare_unigram_mat_no_stop, rare_unigram_words_no_stop)
test_X_sif_rare_uni_no_stop_reduced = reduce_topic_dim(test_X_sif_rare_uni_no_stop, n=200)

test_Z_sif_rare_uni_no_stop = get_hier_cluster_of(test_X_sif_rare_uni_no_stop)

print(' took '+ str(time.process_time() - t_start) + 'seconds')

In [ ]:
#Defining clusters for tfidf and sif
## Hierarchical clustering
test_clusters_tfidf_uni_no_stop = fcluster(test_Z_tfidf_rare_uni_no_stop, k, criterion='maxclust')
test_color_data_tfidf_uni_no_stop, raw_color_keys1 = get_color_data(test_clusters_tfidf_uni_no_stop)

test_clusters_sif_uni_no_stop = fcluster(test_Z_sif_rare_uni_no_stop, k, criterion='maxclust')
test_color_data_tfidf_uni_no_stop, raw_color_keys1 = get_color_data(test_clusters_tfidf_uni_no_stop)

In [ ]:
k=15
print(k)

In [ ]:
test_tsnemat_tfidf_rare_uni_no_stop_50 = tsne_coords(test_X_tfidf_rare_uni_no_stop1)
test_tsnemat_tfidf_rare_uni_no_stop_500 = tsne_coords(test_X_tfidf_rare_uni_no_stop2)
test_tsnemat_sif_rare_uni_no_stop_300 = tsne_coords(test_X_sif_rare_uni_no_stop)
test_tsnemat_sif_rare_uni_no_stop_100 = tsne_coords(test_X_sif_rare_uni_no_stop_reduced)

In [ ]:
t_start = time.process_time()

#test_tsnemat_tfidf_rare_uni_no_stop1 = tsne_coords(test_X_tfidf_rare_uni_no_stop1)
print('25%')
test_clusters_tfidf_uni_no_stop = fcluster(test_Z_tfidf_rare_uni_no_stop, k, criterion='maxclust')
test_color_data_tfidf_uni_no_stop, raw_color_keys1 = get_color_data(test_clusters_tfidf_uni_no_stop)
print('75%')
test_xy_tfidf_uni_no_stop = plot_tsne(test_color_data_tfidf_uni_no_stop, test_tsnemat_tfidf_rare_uni_no_stop_50, plot_name = 'tfidf'+'_no_stop_50 dim')

t_end = time.process_time()
print(str(t_end-t_start)+' seconds' )

In [ ]:
t_start = time.process_time()

#test_tsnemat_tfidf_rare_uni_no_stop2 = tsne_coords(test_X_tfidf_rare_uni_no_stop2)
print('25%')
test_clusters_tfidf_uni_no_stop2 = fcluster(test_Z_tfidf_rare_uni_no_stop, k, criterion='maxclust')
test_color_data_tfidf_uni_no_stop2, raw_color_keys2 = get_color_data(test_clusters_tfidf_uni_no_stop2)
print('75%')
test_xy_tfidf_uni_no_stop = plot_tsne(test_color_data_tfidf_uni_no_stop2, test_tsnemat_tfidf_rare_uni_no_stop2, plot_name = 'tfidf'+'_no_stop')

t_end = time.process_time()
print(str(t_end-t_start)+' seconds' )

In [ ]:
t_start = time.process_time()

#test_tsnemat_sif_rare_uni_no_stop = tsne_coords(test_X_sif_rare_uni_no_stop)
print('25%')
test_clusters_sif_uni_no_stop = fcluster(test_Z_sif_rare_uni_no_stop, k, criterion='maxclust')
test_color_data_sif_uni_no_stop, raw_color_keys3 = get_color_data(test_clusters_sif_uni_no_stop)
print('75%')
test_xy_sif_uni_no_stop = plot_tsne(test_color_data_sif_uni_no_stop, test_tsnemat_sif_rare_uni_no_stop, plot_name='sif'+'_no_stop')

t_end = time.process_time()
print(str(t_end-t_start)+' seconds' )

In [ ]:
test_xy_sif_uni_no_stop = plot_tsne(test_color_data_sif_uni_no_stop, test_tsnemat_sif_rare_uni_no_stop*10, plot_name='sif'+'_no_stop')


In [ ]:
for cnt, row in enumerate(test_X_sif_rare_uni_no_stop):
    if(sum(row) == 0):
        #print(test_X_sif_rare_uni_no_stop[3712])
        print(test_my_text_chunk[0][cnt])
        print(' ')
        print(cnt)
        print(sum(rare_unigram_mat_no_stop[cnt].toarray()[0]))
        print(' ')
        print(test_X_tfidf_rare_uni_no_stop2[cnt])
        print(test_X_sif_rare_uni_no_stop[cnt])
        print(test_X_sif_rare_uni_no_stop_reduced[cnt])
        print(test_tsnemat_sif_rare_uni_no_stop[cnt])
        print(' ')
        print('\n')

In [ ]:
test_tsnemat_sif_rare_uni_no_stop.shape

In [ ]:
import time

#up to 4 arguments at max
def runtime_(fn=None, args=None):
    
    if(fn==None):
        print('No function given.')
        return ;
    
    print(fn.__name__)
    start_time = time.process_time()
    
    try:
        if(len(args) == 1):
            result = fn(args[0])
        elif(len(args) == 2):
            result = fn(args[0], args[1])
        elif(len(args) == 3):
            result = fn(args[0], args[1], args[2])
        elif(len(args) == 4):
            result = fn(args[0], args[1], args[2], args[3])
    except:
        print('Unmatching number of arguments. The ')
        return ;
    
    end_time = time.process_time()
    print(' Average Run Time:'+str(round(end_time-start_time,5)))
    
    return result

In [ ]:
test_X_sif_rare_uni_no_stop.shape

In [ ]:
#3D Plotting Testing!
# ask on stackoverflow how to make the interactive smooth because 10000+, 100+ matrix is going to be laggy w/o opt.
import time
#past_runtimes
start_time = time.process_time()
#test_tsnemat_sif_rare_uni_no_stop_dim2 = tsne_coords,test_X_sif_rare_uni_no_stop, dim=2)
test_tsnemat_sif_rare_uni_no_stop_dim3 = tsne_coords(test_X_sif_rare_uni_no_stop, dim=3)
print('Mid Run Time (sec): '+str(time.process_time() - start_time))
test_tsnemat_sif_rare_uni_no_stop_dim3_reduced = tsne_coords(reduce_topic_dim(test_X_sif_rare_uni_no_stop, n=250), dim=3)
end_time = time.process_time()
print('Total Run Time (sec): '+str(end_time-start_time))

In [ ]:
test_clusters_sif_uni_no_stop = fcluster(test_Z_sif_rare_uni_no_stop, 15, criterion='maxclust')

In [ ]:
test_color_data_sif_uni_no_stop_dim3, raw_color_keys4 = get_color_data(test_clusters_sif_uni_no_stop_dim3)

In [ ]:
test_xy_sif_uni_no_stop_dim3_reduced = plot_tsne(test_color_data_sif_uni_no_stop, test_tsnemat_sif_rare_uni_no_stop_dim3_reduced, plot_name = 'sif'+'_no_stop')


In [ ]:
test_xy_sif_uni_no_stop_dim3 = plot_tsne(test_color_data_sif_uni_no_stop, test_tsnemat_sif_rare_uni_no_stop_dim3, plot_name = 'sif'+'_no_stop')


In [ ]:
test_color_data_sif_uni_no_stop

In [ ]:
annotate_clusters(test_my_text_chunk[0], test_X_sif_rare_uni_no_stop, test_color_data_sif_uni_no_stop)

In [ ]:
def threeDplot():
    %matplotlib notebook
    test_xy_sif_uni_no_stop_dim3 = plot_tsne(test_color_data_sif_uni_no_stop, test_tsnemat_sif_rare_uni_no_stop_dim3, plot_name = 'sif'+'_no_stop')
#threeDplot()

In [ ]:
def print_with_tags(words, tags=None, newLine=True):
    if(tags == None): tags = [i for i,_ in enumerate(words)]
    final_s = ""
    for i, word in enumerate(words):
        final_s += str(tags[i])+ ':' + str(words[i])
        
        if(newLine): final_s += '\n'
        else: final_s += '    '
            
    print(final_s)
    return final_s

In [ ]:
if(DEBUG): print_with_tags(['a',17,'b','c','d','f'], tags=['red','blue','orange','kiwi','pink','navy'], newLine=False)

In [ ]:
def get_max_min(l):
    if(len(l) < 1):
        print('list has no element.')
        return
    
    max, min = None, None
    max_ind, min_ind = None, None
    
    for index, e in enumerate(l):
        if(index==0):
            max, min = e, e
            max_ind, min_ind = index, index
        if(e > max):
            max, max_ind = e, index
        if(e < min):
            min, min_ind = e, index
    
    result_tag = 'max max_ind, min min_ind'.split()
    result = max, max_ind, min, min_ind
    
    print_with_tags(result, tags=result_tag, newLine=False)
    
    return result

In [ ]:
a = 'apple'
print(a

In [ ]:
def count(s, c):
    if(isinstance(s, str)):
        for ch in 

In [ ]:
test_tfidf_xvals_1500 = list(zip(*test_tsnemat_tfidf_rare_uni_no_stop2))[0]
test_tfidf_yvals_1500 = list(zip(*test_tsnemat_tfidf_rare_uni_no_stop2))[1]
get_max_min(test_tfidf_xvals_1500)
get_max_min(test_tfidf_yvals_1500)
print(test_my_text_chunk[1][847])
print(test_X_tfidf_rare_uni_no_stop2[847])
print(test_tfidf_xvals_1500[847])

In [ ]:
print(max(list(zip(*test_tsnemat_tfidf_rare_uni_no_stop2))[0])) 
print(max(list(zip(*test_tsnemat_tfidf_rare_uni_no_stop2))[1]))

In [ ]:
all_zip = find_nearest_neighbors(test_tsnemat_tfidf_rare_uni_no_stop2,(-400, 700), 5, None, test_my_text_chunk[0])
print(all_zip)

In [ ]:
all_zip = find_nearest_neighbors(test_tsnemat_sif_rare_uni_no_stop_dim3, [-1.818, 0.512, 342.268], 3, None, test_my_text_chunk[0])
#test_my_text_chunk[0][847]
print(test_tsnemat_sif_rare_uni_no_stop_dim3[846])
print(all_zip)

In [ ]:
aff =['a','b','b','c','d','d','d']
Counter(aff)

In [ ]:
def annotate_clusters(text, mat, clusters,jk=5):
    cluster_counter = Counter(clusters)
    print(cluster_counter)
    clust_dict = {clust: [] for clust in cluster_counter.keys()}
    for index, clust in enumerate(clusters):
        clust_dict[clust].append(index)
    
    print("\n\n")
    #convert indices into actual data
    #print(clust_dict)
    for key in clust_dict:
        new_val = []
        for v in clust_dict[key]:
            new_val.append(mat[v])
        clust_dict[key] = np.array(new_val)
    
    cnt=0
    for cluster in cluster_counter:
        #if(cnt>3):break
        cnt += 1
        print(cluster)
        avg_centroid = do_average(clust_dict[cluster])
        #print(avg_centroid)
        #print(sum(clust_dict[cluster])/len(clust_dict[cluster]))
        #print((avg_centroid) - sum(clust_dict[cluster])/len(clust_dict[cluster]))
        #all_zip = find_nearest_neighbors(mat, avg_centroid, 3, cluster_counter, text)
        #when you hate 300 dim vectors
        all_zip = find_nearest_neighbors(mat, avg_centroid, jk, cluster_counter, text)
        
        print('Nearest {0} data points: '.format(jk))
        for d in all_zip:
            print(d)
        print('\n')
    return all_zip

In [ ]:
annotate_clusters(test_my_text_chunk[0], test_tsnemat_tfidf_rare_uni_no_stop1, test_color_data_sif_uni_no_stop,jk=15)

In [ ]:
annotate_clusters(test_my_text_chunk[0], test_tsnemat_sif_rare_uni_no_stop, test_color_data_sif_uni_no_stop,jk=15)

In [ ]:
def do_average(embs):
    return reduce(lambda x,y: x+y, embs)/len(embs)

In [ ]:
asdf = [np.array([1,2,3]), np.array([10,20,30]), np.array([7,8,9])]
do_average(asdf)

In [ ]:
def add_dims(mat, add_rows):
    """
    mat: [[1,2,3],[10,20,30],[100,200,300]]
    add_rows = [[-1,-1,-1]]
    """
    if(len(mat) != len(add_rows[0])):
        print('dimensions unmatching or wrong format')
        return ;
    
    return list(zip(*(list(zip(*mat)) + add_rows)))    
    


In [ ]:
add_dims(np.array([[1,2,3,4,5],[10,11,12,13,14],[20,21,22,23,24]]), [[-1,-1,-1]])

In [ ]:
test_X_sif_rare_uni_no_stop[0]

In [ ]:
from scipy import spatial
def find_nearest_neighbors(myArray, x, k, color_data=None, text_data=None):
    myKDTree = spatial.KDTree(myArray)
    #x = myArray[xi]
    #print(len(x))
    nearest_dist, nearest_index = myKDTree.query(x)
    #print(' nearest data point: {0}, and its index: {1}'.format(nearest_dist, nearest_index))
    
    dist, index = myKDTree.query(myArray[nearest_index], k = k)
    
    if color_data: my_color_list = [color_data[ind] for ind in index]
    else: my_color_list = [None for ind in index]
    
    if text_data: my_text_list = [text_data[ind] for ind in index]
    else: my_text_list = [None for ind in index]
        
    all_zip = list(zip(dist,index,my_color_list,my_text_list))
    #all_zip = list(zip(dist,index,myArray[index],my_color_list,my_text_list))
 
    return all_zip

In [ ]:
print(len(test_color_data_sif_uni_no_stop))

In [ ]:
myArray = test_X_sif_rare_uni_no_stop
print(test_my_text_chunk[0][60])
points = find_nearest_point(myArray, 60, 20, color_data=test_color_data_sif_uni_no_stop)


In [ ]:
for p in points:
    print(p)
    print(test_my_text_chunk[0][p[1]])

In [ ]:
print(test_my_text_chunk[0][41])
for cnt, i in enumerate(indexs):
    print(cnt)
    print(test_my_text_chunk[0][i])
    print('\n')

In [ ]:
def find_nearest_k_neighbors(myArray, x,y,k):
    

In [ ]:
color_hashing = lambda x,length: (x*100+977)%length
test_color_data_sif_uni_no_stop = get_color_data(test_clusters_sif_uni_no_stop, color_index_hash=color_hashing)


In [ ]:
test_xy_sif_uni_no_stop = plot_tsne(test_color_data_sif_uni_no_stop, test_tsnemat_sif_rare_uni_no_stop, plot_name='sif'+'_no_stop')


In [ ]:
do_visualize(rare_unigram_mat_no_stop, rare_unigram_words_no_stop, plot_name='_no_stop', k=30, n=200)
